In [ ]:
from torch_geometric.data import Data
from GraphDataBuilder import DataBuilder

In [ ]:
data_builder = DataBuilder('path to turtle', 'NLP_client')

data = Data(x=data_builder.get_x(), edge_index=data_builder.get_edge_index(), ...)